In [1]:
import pandas as pd
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from functools import reduce

os.chdir(r'\\ns-sisci01hd\home_s01\Citrix_Folder_Redirect\Downloads\SPFD')

df = pd.read_pickle('Data_For_Agent_Modelling_0228_v3.pkl')
travel_remove = list(df[df['PLAN_DESC'].str.contains('TRAVEL')]['PLAN_DESC'].unique())
df = df[~df['PLAN_DESC'].isin(travel_remove)]
df = df[df['VAT_ID'] == 'AGNCY']

# defining time period
last_date = pd.to_datetime('2016/12/31')
start_date = pd.to_datetime('2000/01/01')

# filterning for required columns alone
df = df[['AGENT1','CONTRACT_DATE','KEY','INSURED_NAME','ISSUE_DATE','PAID_TO_DATE','STATUS','LOB']]

# Filterning and cleaning the data
df = df[(df['ISSUE_DATE'] >= start_date) & (df['ISSUE_DATE'] <= last_date)]
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

df = df[~df['LOB'].str.startswith('Annuity')]
df = df[~(df['PAID_TO_DATE'] < df['ISSUE_DATE'])]
df.loc[df['LOB'].str.contains('Life'),'LOB'] = 'Life'

# Creating date based columns - year, week, quarter and Agent Experience
df['YEAR'] = df['ISSUE_DATE'].dt.year
df['WEEK_NUM'] = df['ISSUE_DATE'].dt.week
df['QUARTER'] = df['ISSUE_DATE'].dt.quarter
df.loc[df['QUARTER'] == 1, 'QUARTER'] = 'Q1'
df.loc[df['QUARTER'] == 2, 'QUARTER'] = 'Q2'
df.loc[df['QUARTER'] == 3, 'QUARTER'] = 'Q3'
df.loc[df['QUARTER'] == 4, 'QUARTER'] = 'Q4'

df['TENURE'] = (df['PAID_TO_DATE'] - df['ISSUE_DATE']).dt.days
df['TENURE_BIN'] = pd.cut(df['TENURE'],bins=[0,183,366,731,9999999]
                          ,labels = ['lte_6mo','gt_6mo_lte_1ye','gt_1ye_lte_2ye','gt_2ye'])
df['AGENT_EXP'] = (df['ISSUE_DATE'] - df['CONTRACT_DATE']).dt.days

# Identifying the first time a customer came
df = df.reset_index(drop=True)
df = df.sort_values(['INSURED_NAME', 'ISSUE_DATE'])
df['NEW_CUSTOMER'] = df.groupby(['AGENT1', 'LOB', 'INSURED_NAME']).cumcount()+1
df['NEW_CUSTOMER'] = np.where(df['NEW_CUSTOMER'] == 1, 'NewCustomer', 'ExistingCustomer')

# Creating a dataframe with the skeleton for joining
right = df[(df['ISSUE_DATE'].dt.year>=2000)].drop_duplicates()
left = df[(df['ISSUE_DATE'].dt.year>2009)][['AGENT1','LOB','YEAR']].drop_duplicates()

# merging with skeleton to get data at a agent, year level
merge_df = left.merge(right,how='left',on=['AGENT1','LOB']).drop_duplicates()
merge_df['diff_years'] = merge_df['YEAR_x'] - merge_df['YEAR_y']
merge_df = merge_df[(merge_df['diff_years']>=0) & (merge_df['diff_years']<=10)]

merge_df['TimePeriod'] = np.where(merge_df['diff_years']==0,
                                  'CurrentYear', 'Lookback')
merge_df = merge_df.drop(['diff_years','YEAR_y'], axis=1)\
.rename(columns={'YEAR_x':'YEAR'})

# Getting total policies
x = merge_df.groupby(['AGENT1','YEAR','LOB','TimePeriod'])['KEY'].nunique()\
.reset_index().rename(columns={'KEY':'#TotalPolicies'})
total_policies = x.pivot_table(index=['AGENT1','YEAR'],
                               columns=['TimePeriod','LOB'],
                               aggfunc='mean',
                               fill_value = 0, 
                               margins = False)
total_policies.columns = ['_'.join(i[1:]+[i[0]]) for i in (list(i) for i in total_policies.columns)]
total_policies.reset_index(inplace=True)

# Getting total policies for new and existing customers
x = merge_df.groupby(['AGENT1','YEAR','LOB','TimePeriod','NEW_CUSTOMER'])['KEY'].nunique()\
.reset_index().rename(columns={'KEY':'#TotalPolicies'})
total_policies_new_ex = x.pivot_table(index=['AGENT1','YEAR'],
                               columns=['TimePeriod','LOB','NEW_CUSTOMER'],
                               aggfunc='mean',
                               fill_value = 0, 
                               margins = False)
total_policies_new_ex.columns = ['_'.join(i[1:]+[i[0]]) for i in (list(i) for i in total_policies_new_ex.columns)]
total_policies_new_ex.reset_index(inplace=True)

# Getting total policies at a quarter level
x = merge_df.groupby(['AGENT1','YEAR','QUARTER','LOB','TimePeriod'])['KEY'].nunique()\
.reset_index().rename(columns={'KEY':'#TotalPolicies'})
total_policies_quarter = x.pivot_table(index=['AGENT1','YEAR'],
                                       columns=['TimePeriod','QUARTER','LOB'],
                                       aggfunc='mean',
                                       fill_value = 0, 
                                       margins = False)
total_policies_quarter.columns = ['_'.join(i[1:]+[i[0]]) for i in (list(i) for i in total_policies_quarter.columns)]
total_policies_quarter.reset_index(inplace=True)

# getting agent experience at a year level
agent_experience = merge_df.groupby(['AGENT1','YEAR']).agg({'AGENT_EXP':['median','max']})
agent_experience.columns = ['_'.join(i) for i in agent_experience.columns]
agent_experience.reset_index(inplace=True)

# getting policy tenure for an agent for each lob
agent_lob_tenure = merge_df.groupby(['AGENT1','YEAR','LOB']).agg({'TENURE':['mean','std','median']})
agent_lob_tenure.columns = ['_'.join(i) for i in agent_lob_tenure.columns]
agent_lob_tenure = agent_lob_tenure.pivot_table(index=['AGENT1','YEAR'],
                                       columns=['LOB'],
                                       aggfunc='mean',
                                       fill_value = 0, 
                                       margins = False)
agent_lob_tenure.columns = ['_'.join(i) for i in agent_lob_tenure.columns]
agent_lob_tenure.reset_index(inplace=True)
agent_lob_tenure[[i for i in agent_lob_tenure.columns if '_std' in i]].fillna(0,inplace=True)

# getting number of new and existing customers for an agent
x = merge_df.groupby(['AGENT1','YEAR','TimePeriod','LOB','NEW_CUSTOMER'])['INSURED_NAME'].nunique()\
.reset_index().rename(columns={'INSURED_NAME':'#Customers'})
lob_time_customers = x.pivot_table(index=['AGENT1','YEAR'],
                                       columns=['TimePeriod','LOB','NEW_CUSTOMER'],
                                       aggfunc='mean',
                                       fill_value = 0, 
                                       margins = False)
lob_time_customers.columns = ['_'.join(i[1:]+[i[0]]) for i in (list(i) for i in lob_time_customers.columns)]
lob_time_customers.reset_index(inplace=True)

# getting number of new and existing customers for an agent at a quarter level
x = merge_df.groupby(['AGENT1','YEAR','TimePeriod','LOB','NEW_CUSTOMER','QUARTER'])['INSURED_NAME'].nunique()\
.reset_index().rename(columns={'INSURED_NAME':'#Customers'})
lob_time_quarter_customers = x.pivot_table(index=['AGENT1','YEAR'],
                                       columns=['TimePeriod','QUARTER','LOB','NEW_CUSTOMER'],
                                       aggfunc='mean',
                                       fill_value = 0, 
                                       margins = False)
lob_time_quarter_customers.columns = ['_'.join(i[1:]+[i[0]]) for i in (list(i) for i in lob_time_quarter_customers.columns)]
lob_time_quarter_customers.reset_index(inplace=True)

# getting max policies made by the agent in a week
x = merge_df.groupby(['AGENT1','YEAR','TimePeriod','LOB','WEEK_NUM'])['KEY'].nunique().reset_index()
x = x.groupby(['AGENT1','YEAR','TimePeriod','LOB'])['KEY'].max().reset_index()\
.rename(columns={'KEY':'#max_policies_in_a_week'})
lob_time_max_policies = x.pivot_table(index=['AGENT1','YEAR'],
                                       columns=['TimePeriod','LOB'],
                                       aggfunc='mean',
                                       fill_value = 0, 
                                       margins = False)
lob_time_max_policies.columns = ['_'.join(i[1:]+[i[0]]) for i in (list(i) for i in lob_time_max_policies.columns)]
lob_time_max_policies.reset_index(inplace=True)

# getting max policies made by the agent in a week by quarter
x = merge_df.groupby(['AGENT1','YEAR','TimePeriod','LOB','QUARTER','WEEK_NUM'])['KEY'].nunique().reset_index()
x = x.groupby(['AGENT1','YEAR','TimePeriod','QUARTER','LOB'])['KEY'].max().reset_index()\
.rename(columns={'KEY':'#max_policies_in_a_week'})
lob_time_quarter_max_policies = x.pivot_table(index=['AGENT1','YEAR'],
                                       columns=['TimePeriod','QUARTER','LOB'],
                                       aggfunc='mean',
                                       fill_value = 0, 
                                       margins = False)
lob_time_quarter_max_policies.columns = \
['_'.join(i[1:]+[i[0]]) for i in (list(i) for i in lob_time_quarter_max_policies.columns)]
lob_time_quarter_max_policies.reset_index(inplace=True)

# getting no of days the agent was active
x = merge_df.groupby(['AGENT1','YEAR','TimePeriod','LOB'])['ISSUE_DATE'].nunique()\
.reset_index().rename(columns={'ISSUE_DATE':'#Active_days_agent'})
lob_time_active_days= x.pivot_table(index=['AGENT1','YEAR'],
                                       columns=['TimePeriod','LOB'],
                                       aggfunc='mean',
                                       fill_value = 0, 
                                       margins = False)
lob_time_active_days.columns = \
['_'.join(i[1:]+[i[0]]) for i in (list(i) for i in lob_time_active_days.columns)]
lob_time_active_days.reset_index(inplace=True)

# getting no of days the agent was active by quarter
x = merge_df.groupby(['AGENT1','YEAR','TimePeriod','QUARTER','LOB'])['ISSUE_DATE'].nunique()\
.reset_index().rename(columns={'ISSUE_DATE':'#Active_days_agent'})
lob_time_quarter_active_days= x.pivot_table(index=['AGENT1','YEAR'],
                                       columns=['TimePeriod','QUARTER','LOB'],
                                       aggfunc='mean',
                                       fill_value = 0, 
                                       margins = False)
lob_time_quarter_active_days.columns = \
['_'.join(i[1:]+[i[0]]) for i in (list(i) for i in lob_time_quarter_active_days.columns)]
lob_time_quarter_active_days.reset_index(inplace=True)

# subsetting data for lapse calculation
sub_merge_df = merge_df[merge_df['STATUS'].isin(['Terminated - Lapsed no Value','Terminated - Cash Surrender'])]
sub_merge_df = sub_merge_df[sub_merge_df['TimePeriod'] == 'Lookback']
sub_merge_df.loc[sub_merge_df['STATUS']=='Terminated - Lapsed no Value','STATUS'] = 'TLNV'
sub_merge_df.loc[sub_merge_df['STATUS']=='Terminated - Cash Surrender','STATUS'] = 'TCS'

# getting lapsed policies
x = sub_merge_df.groupby(['AGENT1','YEAR','TimePeriod','LOB','STATUS','TENURE_BIN'])['KEY'].nunique()\
.reset_index().rename(columns={'KEY':'#LapsedPolicies'})
lapsed_policies= x.pivot_table(index=['AGENT1','YEAR'],
                                       columns=['TimePeriod','LOB','STATUS','TENURE_BIN'],
                                       aggfunc='mean',
                                       fill_value = 0, 
                                       margins = False)
lapsed_policies.columns = \
['_'.join(i[1:]+[i[0]]) for i in (list(i) for i in lapsed_policies.columns)]
lapsed_policies.reset_index(inplace=True)

# getting lapsed policies by new or existing customer
x = sub_merge_df.groupby(['AGENT1','YEAR','TimePeriod','LOB','STATUS','TENURE_BIN','NEW_CUSTOMER'])['KEY'].nunique()\
.reset_index().rename(columns={'KEY':'#LapsedPolicies'})
lapsed_policies_new_existing = x.pivot_table(index=['AGENT1','YEAR'],
                                       columns=['TimePeriod','LOB','STATUS','TENURE_BIN','NEW_CUSTOMER'],
                                       aggfunc='mean',
                                       fill_value = 0, 
                                       margins = False)
lapsed_policies_new_existing.columns = \
['_'.join(i[1:]+[i[0]]) for i in (list(i) for i in lapsed_policies_new_existing.columns)]
lapsed_policies_new_existing.reset_index(inplace=True)

# merging the DF's into one
all_dfs = [total_policies,total_policies_new_ex,total_policies_quarter,lob_time_active_days,
           lob_time_quarter_active_days,agent_experience,agent_lob_tenure,
           lob_time_customers,lob_time_quarter_customers,lob_time_max_policies,
           lob_time_quarter_max_policies,lapsed_policies,lapsed_policies_new_existing]

final_df = reduce(lambda left, right: pd.merge(left, right, how='left',
                                                on=['AGENT1', 
                                                    'YEAR']),
                                                all_dfs)

# calculating percentage metrics - total policies quarter share
q = ['CurrentYear_Q1_Accident & Health_#TotalPolicies',
       'CurrentYear_Q1_Life_#TotalPolicies',
       'CurrentYear_Q1_Unit Linked_#TotalPolicies',
       'CurrentYear_Q2_Accident & Health_#TotalPolicies',
       'CurrentYear_Q2_Life_#TotalPolicies',
       'CurrentYear_Q2_Unit Linked_#TotalPolicies',
       'CurrentYear_Q3_Accident & Health_#TotalPolicies',
       'CurrentYear_Q3_Life_#TotalPolicies',
       'CurrentYear_Q3_Unit Linked_#TotalPolicies',
       'CurrentYear_Q4_Accident & Health_#TotalPolicies',
       'CurrentYear_Q4_Life_#TotalPolicies',
       'CurrentYear_Q4_Unit Linked_#TotalPolicies',
       'Lookback_Q1_Accident & Health_#TotalPolicies',
       'Lookback_Q1_Life_#TotalPolicies',
       'Lookback_Q1_Unit Linked_#TotalPolicies',
       'Lookback_Q2_Accident & Health_#TotalPolicies',
       'Lookback_Q2_Life_#TotalPolicies',
       'Lookback_Q2_Unit Linked_#TotalPolicies',
       'Lookback_Q3_Accident & Health_#TotalPolicies',
       'Lookback_Q3_Life_#TotalPolicies',
       'Lookback_Q3_Unit Linked_#TotalPolicies',
       'Lookback_Q4_Accident & Health_#TotalPolicies',
       'Lookback_Q4_Life_#TotalPolicies',
       'Lookback_Q4_Unit Linked_#TotalPolicies']

t = [  'CurrentYear_Accident & Health_#TotalPolicies',
       'CurrentYear_Life_#TotalPolicies',
       'CurrentYear_Unit Linked_#TotalPolicies',
       'CurrentYear_Accident & Health_#TotalPolicies',
       'CurrentYear_Life_#TotalPolicies',
       'CurrentYear_Unit Linked_#TotalPolicies',
     'CurrentYear_Accident & Health_#TotalPolicies',
       'CurrentYear_Life_#TotalPolicies',
       'CurrentYear_Unit Linked_#TotalPolicies',
     'CurrentYear_Accident & Health_#TotalPolicies',
       'CurrentYear_Life_#TotalPolicies',
       'CurrentYear_Unit Linked_#TotalPolicies',
       'Lookback_Accident & Health_#TotalPolicies',
       'Lookback_Life_#TotalPolicies',
       'Lookback_Unit Linked_#TotalPolicies',
    'Lookback_Accident & Health_#TotalPolicies',
       'Lookback_Life_#TotalPolicies',
       'Lookback_Unit Linked_#TotalPolicies',
    'Lookback_Accident & Health_#TotalPolicies',
       'Lookback_Life_#TotalPolicies',
       'Lookback_Unit Linked_#TotalPolicies',
    'Lookback_Accident & Health_#TotalPolicies',
       'Lookback_Life_#TotalPolicies',
       'Lookback_Unit Linked_#TotalPolicies']

for i in zip(q,t):
    final_df[i[0]] = final_df[i[0]]/final_df[i[1]]
    final_df[i[0]].fillna(0,inplace=True)

# calculating percentage metrics - policies per active day
q = ['CurrentYear_Accident & Health_#Active_days_agent',
       'CurrentYear_Life_#Active_days_agent',
       'CurrentYear_Unit Linked_#Active_days_agent',
       'Lookback_Accident & Health_#Active_days_agent',
       'Lookback_Life_#Active_days_agent',
       'Lookback_Unit Linked_#Active_days_agent']

t = ['CurrentYear_Accident & Health_#TotalPolicies',
       'CurrentYear_Life_#TotalPolicies',
       'CurrentYear_Unit Linked_#TotalPolicies',
       'Lookback_Accident & Health_#TotalPolicies',
       'Lookback_Life_#TotalPolicies',
       'Lookback_Unit Linked_#TotalPolicies']

n = ['CurrentYear_Accident & Health_#avg_policy_per_active_day',
       'CurrentYear_Life_#avg_policy_per_active_day',
       'CurrentYear_Unit Linked_#avg_policy_per_active_day',
       'Lookback_Accident & Health_#avg_policy_per_active_day',
       'Lookback_Life_#avg_policy_per_active_day',
       'Lookback_Unit Linked_#avg_policy_per_active_day']

for i in zip(q,t,n):
    final_df[i[2]] = final_df[i[1]]/final_df[i[0]]
    final_df[i[2]].fillna(0,inplace=True)

# calculating percentage metrics - policies per active day by quarter
q = ['CurrentYear_Q1_Accident & Health_#Active_days_agent',
       'CurrentYear_Q1_Life_#Active_days_agent',
       'CurrentYear_Q1_Unit Linked_#Active_days_agent',
       'CurrentYear_Q2_Accident & Health_#Active_days_agent',
       'CurrentYear_Q2_Life_#Active_days_agent',
       'CurrentYear_Q2_Unit Linked_#Active_days_agent',
       'CurrentYear_Q3_Accident & Health_#Active_days_agent',
       'CurrentYear_Q3_Life_#Active_days_agent',
       'CurrentYear_Q3_Unit Linked_#Active_days_agent',
       'CurrentYear_Q4_Accident & Health_#Active_days_agent',
       'CurrentYear_Q4_Life_#Active_days_agent',
       'CurrentYear_Q4_Unit Linked_#Active_days_agent',
       'Lookback_Q1_Accident & Health_#Active_days_agent',
       'Lookback_Q1_Life_#Active_days_agent',
       'Lookback_Q1_Unit Linked_#Active_days_agent',
       'Lookback_Q2_Accident & Health_#Active_days_agent',
       'Lookback_Q2_Life_#Active_days_agent',
       'Lookback_Q2_Unit Linked_#Active_days_agent',
       'Lookback_Q3_Accident & Health_#Active_days_agent',
       'Lookback_Q3_Life_#Active_days_agent',
       'Lookback_Q3_Unit Linked_#Active_days_agent',
       'Lookback_Q4_Accident & Health_#Active_days_agent',
       'Lookback_Q4_Life_#Active_days_agent',
       'Lookback_Q4_Unit Linked_#Active_days_agent']

t = ['CurrentYear_Q1_Accident & Health_#TotalPolicies',
       'CurrentYear_Q1_Life_#TotalPolicies',
       'CurrentYear_Q1_Unit Linked_#TotalPolicies',
       'CurrentYear_Q2_Accident & Health_#TotalPolicies',
       'CurrentYear_Q2_Life_#TotalPolicies',
       'CurrentYear_Q2_Unit Linked_#TotalPolicies',
       'CurrentYear_Q3_Accident & Health_#TotalPolicies',
       'CurrentYear_Q3_Life_#TotalPolicies',
       'CurrentYear_Q3_Unit Linked_#TotalPolicies',
       'CurrentYear_Q4_Accident & Health_#TotalPolicies',
       'CurrentYear_Q4_Life_#TotalPolicies',
       'CurrentYear_Q4_Unit Linked_#TotalPolicies',
       'Lookback_Q1_Accident & Health_#TotalPolicies',
       'Lookback_Q1_Life_#TotalPolicies',
       'Lookback_Q1_Unit Linked_#TotalPolicies',
       'Lookback_Q2_Accident & Health_#TotalPolicies',
       'Lookback_Q2_Life_#TotalPolicies',
       'Lookback_Q2_Unit Linked_#TotalPolicies',
       'Lookback_Q3_Accident & Health_#TotalPolicies',
       'Lookback_Q3_Life_#TotalPolicies',
       'Lookback_Q3_Unit Linked_#TotalPolicies',
       'Lookback_Q4_Accident & Health_#TotalPolicies',
       'Lookback_Q4_Life_#TotalPolicies',
       'Lookback_Q4_Unit Linked_#TotalPolicies']

n = ['CurrentYear_Q1_Accident & Health_#avg_policy_per_active_day',
       'CurrentYear_Q1_Life_#avg_policy_per_active_day',
       'CurrentYear_Q1_Unit Linked_#avg_policy_per_active_day',
       'CurrentYear_Q2_Accident & Health_#avg_policy_per_active_day',
       'CurrentYear_Q2_Life_#avg_policy_per_active_day',
       'CurrentYear_Q2_Unit Linked_#avg_policy_per_active_day',
       'CurrentYear_Q3_Accident & Health_#avg_policy_per_active_day',
       'CurrentYear_Q3_Life_#avg_policy_per_active_day',
       'CurrentYear_Q3_Unit Linked_#avg_policy_per_active_day',
       'CurrentYear_Q4_Accident & Health_#avg_policy_per_active_day',
       'CurrentYear_Q4_Life_#avg_policy_per_active_day',
       'CurrentYear_Q4_Unit Linked_#avg_policy_per_active_day',
       'Lookback_Q1_Accident & Health_#avg_policy_per_active_day',
       'Lookback_Q1_Life_#avg_policy_per_active_day',
       'Lookback_Q1_Unit Linked_#avg_policy_per_active_day',
       'Lookback_Q2_Accident & Health_#avg_policy_per_active_day',
       'Lookback_Q2_Life_#avg_policy_per_active_day',
       'Lookback_Q2_Unit Linked_#avg_policy_per_active_day',
       'Lookback_Q3_Accident & Health_#avg_policy_per_active_day',
       'Lookback_Q3_Life_#avg_policy_per_active_day',
       'Lookback_Q3_Unit Linked_#avg_policy_per_active_day',
       'Lookback_Q4_Accident & Health_#avg_policy_per_active_day',
       'Lookback_Q4_Life_#avg_policy_per_active_day',
       'Lookback_Q4_Unit Linked_#avg_policy_per_active_day']

for i in zip(q,t,n):
    final_df[i[2]] = final_df[i[1]]/final_df[i[0]]
    final_df[i[2]].fillna(0,inplace=True)

# calculating percentage metrics - new customer share by quarter
q = ['CurrentYear_Q1_Accident & Health_ExistingCustomer_#Customers',
       'CurrentYear_Q1_Accident & Health_NewCustomer_#Customers',
       'CurrentYear_Q1_Life_ExistingCustomer_#Customers',
       'CurrentYear_Q1_Life_NewCustomer_#Customers',
       'CurrentYear_Q1_Unit Linked_ExistingCustomer_#Customers',
       'CurrentYear_Q1_Unit Linked_NewCustomer_#Customers',
       'CurrentYear_Q2_Accident & Health_ExistingCustomer_#Customers',
       'CurrentYear_Q2_Accident & Health_NewCustomer_#Customers',
       'CurrentYear_Q2_Life_ExistingCustomer_#Customers',
       'CurrentYear_Q2_Life_NewCustomer_#Customers',
       'CurrentYear_Q2_Unit Linked_ExistingCustomer_#Customers',
       'CurrentYear_Q2_Unit Linked_NewCustomer_#Customers',
       'CurrentYear_Q3_Accident & Health_ExistingCustomer_#Customers',
       'CurrentYear_Q3_Accident & Health_NewCustomer_#Customers',
       'CurrentYear_Q3_Life_ExistingCustomer_#Customers',
       'CurrentYear_Q3_Life_NewCustomer_#Customers',
       'CurrentYear_Q3_Unit Linked_ExistingCustomer_#Customers',
       'CurrentYear_Q3_Unit Linked_NewCustomer_#Customers',
       'CurrentYear_Q4_Accident & Health_ExistingCustomer_#Customers',
       'CurrentYear_Q4_Accident & Health_NewCustomer_#Customers',
       'CurrentYear_Q4_Life_ExistingCustomer_#Customers',
       'CurrentYear_Q4_Life_NewCustomer_#Customers',
       'CurrentYear_Q4_Unit Linked_ExistingCustomer_#Customers',
       'CurrentYear_Q4_Unit Linked_NewCustomer_#Customers',
       'Lookback_Q1_Accident & Health_ExistingCustomer_#Customers',
       'Lookback_Q1_Accident & Health_NewCustomer_#Customers',
       'Lookback_Q1_Life_ExistingCustomer_#Customers',
       'Lookback_Q1_Life_NewCustomer_#Customers',
       'Lookback_Q1_Unit Linked_ExistingCustomer_#Customers',
       'Lookback_Q1_Unit Linked_NewCustomer_#Customers',
       'Lookback_Q2_Accident & Health_ExistingCustomer_#Customers',
       'Lookback_Q2_Accident & Health_NewCustomer_#Customers',
       'Lookback_Q2_Life_ExistingCustomer_#Customers',
       'Lookback_Q2_Life_NewCustomer_#Customers',
       'Lookback_Q2_Unit Linked_ExistingCustomer_#Customers',
       'Lookback_Q2_Unit Linked_NewCustomer_#Customers',
       'Lookback_Q3_Accident & Health_ExistingCustomer_#Customers',
       'Lookback_Q3_Accident & Health_NewCustomer_#Customers',
       'Lookback_Q3_Life_ExistingCustomer_#Customers',
       'Lookback_Q3_Life_NewCustomer_#Customers',
       'Lookback_Q3_Unit Linked_ExistingCustomer_#Customers',
       'Lookback_Q3_Unit Linked_NewCustomer_#Customers',
       'Lookback_Q4_Accident & Health_ExistingCustomer_#Customers',
       'Lookback_Q4_Accident & Health_NewCustomer_#Customers',
       'Lookback_Q4_Life_ExistingCustomer_#Customers',
       'Lookback_Q4_Life_NewCustomer_#Customers',
       'Lookback_Q4_Unit Linked_ExistingCustomer_#Customers',
       'Lookback_Q4_Unit Linked_NewCustomer_#Customers']

t = ['CurrentYear_Accident & Health_ExistingCustomer_#Customers',
       'CurrentYear_Accident & Health_NewCustomer_#Customers',
       'CurrentYear_Life_ExistingCustomer_#Customers',
       'CurrentYear_Life_NewCustomer_#Customers',
       'CurrentYear_Unit Linked_ExistingCustomer_#Customers',
       'CurrentYear_Unit Linked_NewCustomer_#Customers',
 'CurrentYear_Accident & Health_ExistingCustomer_#Customers',
       'CurrentYear_Accident & Health_NewCustomer_#Customers',
       'CurrentYear_Life_ExistingCustomer_#Customers',
       'CurrentYear_Life_NewCustomer_#Customers',
       'CurrentYear_Unit Linked_ExistingCustomer_#Customers',
       'CurrentYear_Unit Linked_NewCustomer_#Customers',
 'CurrentYear_Accident & Health_ExistingCustomer_#Customers',
       'CurrentYear_Accident & Health_NewCustomer_#Customers',
       'CurrentYear_Life_ExistingCustomer_#Customers',
       'CurrentYear_Life_NewCustomer_#Customers',
       'CurrentYear_Unit Linked_ExistingCustomer_#Customers',
       'CurrentYear_Unit Linked_NewCustomer_#Customers',
 'CurrentYear_Accident & Health_ExistingCustomer_#Customers',
       'CurrentYear_Accident & Health_NewCustomer_#Customers',
       'CurrentYear_Life_ExistingCustomer_#Customers',
       'CurrentYear_Life_NewCustomer_#Customers',
       'CurrentYear_Unit Linked_ExistingCustomer_#Customers',
       'CurrentYear_Unit Linked_NewCustomer_#Customers',
       'Lookback_Accident & Health_ExistingCustomer_#Customers',
       'Lookback_Accident & Health_NewCustomer_#Customers',
       'Lookback_Life_ExistingCustomer_#Customers',
       'Lookback_Life_NewCustomer_#Customers',
       'Lookback_Unit Linked_ExistingCustomer_#Customers',
       'Lookback_Unit Linked_NewCustomer_#Customers',
'Lookback_Accident & Health_ExistingCustomer_#Customers',
       'Lookback_Accident & Health_NewCustomer_#Customers',
       'Lookback_Life_ExistingCustomer_#Customers',
       'Lookback_Life_NewCustomer_#Customers',
       'Lookback_Unit Linked_ExistingCustomer_#Customers',
       'Lookback_Unit Linked_NewCustomer_#Customers',
'Lookback_Accident & Health_ExistingCustomer_#Customers',
       'Lookback_Accident & Health_NewCustomer_#Customers',
       'Lookback_Life_ExistingCustomer_#Customers',
       'Lookback_Life_NewCustomer_#Customers',
       'Lookback_Unit Linked_ExistingCustomer_#Customers',
       'Lookback_Unit Linked_NewCustomer_#Customers',
'Lookback_Accident & Health_ExistingCustomer_#Customers',
       'Lookback_Accident & Health_NewCustomer_#Customers',
       'Lookback_Life_ExistingCustomer_#Customers',
       'Lookback_Life_NewCustomer_#Customers',
       'Lookback_Unit Linked_ExistingCustomer_#Customers',
       'Lookback_Unit Linked_NewCustomer_#Customers']

for i in zip(q,t):
    final_df[i[0]] = final_df[i[0]]/final_df[i[1]]
    final_df[i[0]].fillna(0,inplace=True)

# calculating percentage metrics - lapse rate
q =['Lookback_Accident & Health_TCS_gt_1ye_lte_2ye_#LapsedPolicies',
       'Lookback_Accident & Health_TCS_gt_2ye_#LapsedPolicies',
       'Lookback_Accident & Health_TCS_gt_6mo_lte_1ye_#LapsedPolicies',
       'Lookback_Accident & Health_TCS_lte_6mo_#LapsedPolicies',
       'Lookback_Accident & Health_TLNV_gt_1ye_lte_2ye_#LapsedPolicies',
       'Lookback_Accident & Health_TLNV_gt_2ye_#LapsedPolicies',
       'Lookback_Accident & Health_TLNV_gt_6mo_lte_1ye_#LapsedPolicies',
       'Lookback_Accident & Health_TLNV_lte_6mo_#LapsedPolicies',
       'Lookback_Life_TCS_gt_1ye_lte_2ye_#LapsedPolicies',
       'Lookback_Life_TCS_gt_2ye_#LapsedPolicies',
       'Lookback_Life_TCS_gt_6mo_lte_1ye_#LapsedPolicies',
       'Lookback_Life_TCS_lte_6mo_#LapsedPolicies',
       'Lookback_Life_TLNV_gt_1ye_lte_2ye_#LapsedPolicies',
       'Lookback_Life_TLNV_gt_2ye_#LapsedPolicies',
       'Lookback_Life_TLNV_gt_6mo_lte_1ye_#LapsedPolicies',
       'Lookback_Life_TLNV_lte_6mo_#LapsedPolicies',
       'Lookback_Unit Linked_TCS_gt_1ye_lte_2ye_#LapsedPolicies',
       'Lookback_Unit Linked_TCS_gt_2ye_#LapsedPolicies',
       'Lookback_Unit Linked_TCS_gt_6mo_lte_1ye_#LapsedPolicies',
       'Lookback_Unit Linked_TCS_lte_6mo_#LapsedPolicies',
       'Lookback_Unit Linked_TLNV_gt_1ye_lte_2ye_#LapsedPolicies',
       'Lookback_Unit Linked_TLNV_gt_2ye_#LapsedPolicies',
       'Lookback_Unit Linked_TLNV_gt_6mo_lte_1ye_#LapsedPolicies',
       'Lookback_Unit Linked_TLNV_lte_6mo_#LapsedPolicies']

t = []
t.extend(['Lookback_Accident & Health_#TotalPolicies']*8)
t.extend(['Lookback_Life_#TotalPolicies']*8)
t.extend(['Lookback_Unit Linked_#TotalPolicies']*8)

for i in zip(q,t):
    final_df[i[0]] = final_df[i[0]]/final_df[i[1]]
    final_df[i[0]].fillna(0,inplace=True)

# calculating percentage metrics - lapse rate by existing vs new
q = [ 'Lookback_Accident & Health_TCS_gt_1ye_lte_2ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TCS_gt_1ye_lte_2ye_NewCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TCS_gt_2ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TCS_gt_2ye_NewCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TCS_gt_6mo_lte_1ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TCS_gt_6mo_lte_1ye_NewCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TCS_lte_6mo_ExistingCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TCS_lte_6mo_NewCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TLNV_gt_1ye_lte_2ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TLNV_gt_1ye_lte_2ye_NewCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TLNV_gt_2ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TLNV_gt_2ye_NewCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TLNV_gt_6mo_lte_1ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TLNV_gt_6mo_lte_1ye_NewCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TLNV_lte_6mo_ExistingCustomer_#LapsedPolicies',
       'Lookback_Accident & Health_TLNV_lte_6mo_NewCustomer_#LapsedPolicies',
       'Lookback_Life_TCS_gt_1ye_lte_2ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Life_TCS_gt_1ye_lte_2ye_NewCustomer_#LapsedPolicies',
       'Lookback_Life_TCS_gt_2ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Life_TCS_gt_2ye_NewCustomer_#LapsedPolicies',
       'Lookback_Life_TCS_gt_6mo_lte_1ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Life_TCS_gt_6mo_lte_1ye_NewCustomer_#LapsedPolicies',
       'Lookback_Life_TCS_lte_6mo_ExistingCustomer_#LapsedPolicies',
       'Lookback_Life_TCS_lte_6mo_NewCustomer_#LapsedPolicies',
       'Lookback_Life_TLNV_gt_1ye_lte_2ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Life_TLNV_gt_1ye_lte_2ye_NewCustomer_#LapsedPolicies',
       'Lookback_Life_TLNV_gt_2ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Life_TLNV_gt_2ye_NewCustomer_#LapsedPolicies',
       'Lookback_Life_TLNV_gt_6mo_lte_1ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Life_TLNV_gt_6mo_lte_1ye_NewCustomer_#LapsedPolicies',
       'Lookback_Life_TLNV_lte_6mo_ExistingCustomer_#LapsedPolicies',
       'Lookback_Life_TLNV_lte_6mo_NewCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TCS_gt_1ye_lte_2ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TCS_gt_1ye_lte_2ye_NewCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TCS_gt_2ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TCS_gt_2ye_NewCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TCS_gt_6mo_lte_1ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TCS_gt_6mo_lte_1ye_NewCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TCS_lte_6mo_ExistingCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TCS_lte_6mo_NewCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TLNV_gt_1ye_lte_2ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TLNV_gt_1ye_lte_2ye_NewCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TLNV_gt_2ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TLNV_gt_2ye_NewCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TLNV_gt_6mo_lte_1ye_ExistingCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TLNV_gt_6mo_lte_1ye_NewCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TLNV_lte_6mo_ExistingCustomer_#LapsedPolicies',
       'Lookback_Unit Linked_TLNV_lte_6mo_NewCustomer_#LapsedPolicies']

t = []
t.extend(['Lookback_Accident & Health_ExistingCustomer_#TotalPolicies',
       'Lookback_Accident & Health_NewCustomer_#TotalPolicies']*8)
t.extend(['Lookback_Life_ExistingCustomer_#TotalPolicies',
       'Lookback_Life_NewCustomer_#TotalPolicies']*8)
t.extend(['Lookback_Unit Linked_ExistingCustomer_#TotalPolicies',
       'Lookback_Unit Linked_NewCustomer_#TotalPolicies']*8)

for i in zip(q,t):
    final_df[i[0]] = final_df[i[0]]/final_df[i[1]]
    final_df[i[0]].fillna(0,inplace=True)

# removing agents who've sold less than 20 policies in the lookback period
# and less than 15 in the current year
filter_a = merge_df.groupby(['AGENT1','YEAR','TimePeriod'])['KEY'].nunique().reset_index()
filter_a = filter_a.pivot_table(index=['AGENT1', 'YEAR'],
                                            columns=['TimePeriod'], values = ['KEY'], fill_value=0,
                                            margins=False, aggfunc='mean')
filter_a.columns = [('_').join(col) for col in filter_a.columns]
filter_a.reset_index(inplace = True)
filter_a = filter_a[(filter_a['KEY_CurrentYear']>=15) & (filter_a['KEY_Lookback']>=20)]
filter_a = filter_a.drop(['KEY_CurrentYear','KEY_Lookback'], axis=1).drop_duplicates()

final_df = final_df.merge(filter_a,how='inner',on=['AGENT1','YEAR'])

t = [i for i in final_df if '_#Customers' in i]
t.extend([i for i in final_df if '_ExistingCustomer' in i and '#TotalPolicies' in i])
t.extend([i for i in final_df if '#Active_days_agent' in i])
t.extend([i for i in final_df if '_Q' in i and '#TotalPolicies' not in i])
t.extend([i for i in final_df if 'week' in i and 'Current' not in i])
t.extend([i for i in final_df if '_Accident & Health' in i and 'TCS' in i])
t.extend([i for i in final_df if 'TENURE' in i and 'mean' not in i])
t.extend(['AGENT_EXP_median'])

final_df_1 = final_df.drop(set(t), axis=1)

q = np.sort([i for i in final_df_1 if '_New' in i and '#Lapsed' not in i])
t = np.sort([i for i in final_df_1 if '#TotalPolicies' in i and 'Customer_' not in i and '_Q' not in i])

for i in zip(q,t):
    final_df_1[i[0]] = final_df_1[i[0]]/final_df_1[i[1]]
    final_df_1[i[0]].fillna(0,inplace=True)
    
q = np.sort([i for i in final_df_1 if 'per_active_day' in i and 'Lookback' not in i])
t = np.sort([i for i in final_df_1 if 'per_active_day' in i and 'Lookback' in i])
n = ['CurrentYear_by_Lkbk_Accident & Health_#avg_policy_per_active_day',
       'CurrentYear_by_Lkbk_Life_#avg_policy_per_active_day',
       'CurrentYear_by_Lkbk_Unit Linked_#avg_policy_per_active_day']

for i in zip(q,t,n):
    final_df_1[i[2]] = final_df_1[i[0]]/final_df_1[i[1]]
    final_df_1[i[2]].fillna(0,inplace=True)
    
final_df_1 = final_df_1.drop(t,axis =1)

final_df_1.fillna(0,inplace=True)
final_df_1.replace([np.inf, -np.inf], 0,inplace=True)
final_df_1 = final_df_1[final_df_1['YEAR']>2009].reset_index(drop=True)

In [111]:
def descriptive_stats(data, source_name=""):
    stats = pd.DataFrame(columns=["Attribute_Name","Data_Type",
                                  "Total_entries",'Zeros','NonZeros','NZ%'])
    stats["Attribute_Name"] = data.dtypes.index.values
    stats["Data_Type"] = data.dtypes.values
    stats["Data_Type"] = stats["Data_Type"].astype(str)
    stats["Data_Type"] =  stats["Data_Type"].map({"int64":"numeric","object":"categorical","float64":"numeric"})
    stats.set_index("Attribute_Name",inplace=True)
    stats["Total_entries"] = data.shape[0]
    stats['Zeros'] = (data == 0).astype(int).sum(axis=0)
    stats['NonZeros'] = stats['Total_entries'] - stats['Zeros']
    stats['NZ%'] = stats['NonZeros']/stats['Total_entries']
    return stats

descriptive_stats(final_df_1).to_excel('OverallModel_FillRate_2.xlsx')

In [112]:
low_fill_rate = list(descriptive_stats(final_df_1)[descriptive_stats(final_df_1)['NZ%']<0.34].index)

In [113]:
final_df_1.drop(low_fill_rate,axis=1,inplace=True)

In [114]:
len(final_df_1),final_df_1['AGENT1'].nunique()

(1082, 351)

In [115]:
final_df_1.head()

AGENT1  YEAR  CurrentYear_Accident & Health_#TotalPolicies  \
0  011199002  2010                                            23   
1  011199002  2011                                            12   
2  011199002  2012                                            34   
3  011199002  2013                                            32   
4  011199002  2014                                             7   

   CurrentYear_Life_#TotalPolicies  CurrentYear_Unit Linked_#TotalPolicies  \
0                               55                                      29   
1                               45                                      28   
2                               41                                      25   
3                               96                                      21   
4                               44                                      26   

   Lookback_Accident & Health_#TotalPolicies  Lookback_Life_#TotalPolicies  \
0                                        436                           540   
1                                        347                           522   
2                                        267                           462   
3                                        266                           478   
4                                        264                           526   

   Lookback_Unit Linked_#TotalPolicies  \
0                                  185   
1                                  207   
2                                  224   
3                                  244   
4                                  261   

   CurrentYear_Accident & Health_NewCustomer_#TotalPolicies  \
0                                           0.521739          
1                                           0.833333          
2                                           0.647059          
3                                           0.562500          
4                                           0.571429          

   CurrentYear_Life_NewCustomer_#TotalPolicies  \
0                                     0.890909   
1                                     0.511111   
2                                     0.804878   
3                                     0.927083   
4                                     0.977273   

                              ...                              \
0                             ...                               
1                             ...                               
2                             ...                               
3                             ...                               
4                             ...                               

   Lookback_Unit Linked_TLNV_lte_6mo_NewCustomer_#LapsedPolicies  \
0                                                0.0               
1                                                0.0               
2                                                0.0               
3                                                0.0               
4                                                0.0               

   Lookback_Unit Linked_TLNV_gt_6mo_lte_1ye_NewCustomer_#LapsedPolicies  \
0                                           0.012987                      
1                                           0.011494                      
2                                           0.021505                      
3                                           0.030457                      
4                                           0.029126                      

   Lookback_Unit Linked_TLNV_gt_1ye_lte_2ye_NewCustomer_#LapsedPolicies  \
0                                           0.051948                      
1                                           0.045977                      
2                                           0.043011                      
3                                           0.055838                      
4                                           0.058252                      

   Lookback_Unit Linked_

In [131]:
final_df_1.to_pickle('OMD_0703.pkl')

In [166]:
os.getcwd()

'\\\\ns-sisci01hd\\home_s01\\Citrix_Folder_Redirect\\Downloads\\SPFD'

In [96]:
profile = pd.read_excel('aklpe_0703_alkpe_results.xlsx',dtype={'AGENT1': object})

In [3]:
profile['Anomalous'] = profile['check'].apply(lambda x: 1 if x>2 else 0)

In [4]:
profile = profile.groupby('AGENT1')['Anomalous'].sum().reset_index()
profile = profile[profile['Anomalous']>0][['AGENT1']]

In [5]:
pro = merge_df.merge(profile,how='inner',on=['AGENT1'])

In [8]:
tot = pro[(pro['TimePeriod']!='Lookback') & (pro['YEAR']>2009)].groupby(['AGENT1','YEAR','LOB'])['KEY']\
.nunique().reset_index().rename(columns={'KEY':'TotalPolicies_current'})

In [10]:
tot_quarter = pro[(pro['TimePeriod']!='Lookback') & (pro['YEAR']>2009)].groupby(['AGENT1','YEAR','QUARTER','LOB'])['KEY']\
.nunique().reset_index().rename(columns={'KEY':'TotalPolicies_current'})
tot_quarter = tot_quarter.pivot_table(index=['AGENT1','YEAR','LOB'],
                                      columns='QUARTER',
                                      aggfunc='mean',
                                      fill_value=0)
tot_quarter.columns = ['_'.join(i) for i in tot_quarter.columns]
tot_quarter = tot_quarter.reset_index()

In [11]:
lapsed = pro[pro['STATUS'].isin(['Terminated - Lapsed no Value','Terminated - Cash Surrender'])\
            & (pro['TimePeriod']!='Lookback') & (pro['YEAR']>2009)]\
.groupby(['AGENT1','YEAR','LOB'])['KEY'].nunique().reset_index().rename(columns={'KEY':'LapsedPolicies'})

In [12]:
active_days = pro[(pro['TimePeriod']!='Lookback') & (pro['YEAR']>2009)]\
.groupby(['AGENT1','YEAR','LOB'])['ISSUE_DATE'].nunique().reset_index()\
.rename(columns={'ISSUE_DATE':'ActiveDays'})

In [13]:
new_insured = pro[(pro['NEW_CUSTOMER']=='NewCustomer') & (pro['TimePeriod']!='Lookback')\
                 & (pro['YEAR']>2009)]\
.groupby(['AGENT1','YEAR','LOB'])['KEY'].nunique()\
.reset_index().rename(columns={'KEY':'NewCustomerPolicies'})

In [14]:
max_pols_in_week  = pro[(pro['TimePeriod']!='Lookback') & (pro['YEAR']>2009)]\
.groupby(['AGENT1','YEAR','WEEK_NUM','LOB'])['KEY'].nunique().reset_index()
max_pols_in_week = max_pols_in_week.groupby(['AGENT1','YEAR','LOB'])['KEY'].max().reset_index()\
.rename(columns={'KEY':'#max_policies_in_a_week'})

In [15]:
pol_not_taken = pro[(pro['STATUS']=='Policy not taken') & (pro['TimePeriod']!='Lookback')\
                   & (pro['YEAR']>2009)]\
.groupby(['AGENT1','YEAR','LOB'])['KEY'].nunique()\
.reset_index().rename(columns={'KEY':'NotTakenPolicies'})

In [16]:
lkbk_pol = pro[(pro['TimePeriod']=='Lookback') & (pro['YEAR']>2009)].groupby(['AGENT1','YEAR','LOB'])['KEY']\
.nunique().reset_index().rename(columns={'KEY':'TotalPolicies_lookback'})

In [17]:
lkbk_lapsed = pro[pro['STATUS'].isin(['Terminated - Lapsed no Value','Terminated - Cash Surrender'])\
            & (pro['TimePeriod']=='Lookback') & (pro['YEAR']>2009)]\
.groupby(['AGENT1','YEAR','TENURE_BIN','LOB'])['KEY'].nunique().reset_index().rename(columns={'KEY':'LapsedPolicies_lookback'})
lkbk_lapsed = lkbk_lapsed.pivot_table(index=['AGENT1','YEAR','LOB'],columns='TENURE_BIN',aggfunc='mean',fill_value=0)
lkbk_lapsed.columns = ['_'.join(i) for i in lkbk_lapsed.columns]
lkbk_lapsed = lkbk_lapsed.reset_index()

In [26]:
all_dfs = [tot,tot_quarter,lapsed,active_days,new_insured,max_pols_in_week,pol_not_taken,lkbk_pol,lkbk_lapsed]

final_df = reduce(lambda left, right: pd.merge(left, right, how='left',
                                                on=['AGENT1', 
                                                    'YEAR','LOB']),
                                                all_dfs)
final_df.fillna(0,inplace = True)
final_df['pol_per_day'] = final_df['TotalPolicies_current']/final_df['ActiveDays']
final_df['pct_new_insured'] = final_df['NewCustomerPolicies']/final_df['TotalPolicies_current']
final_df['LapsedPolicies_lookback_lte_6mo'] = final_df['LapsedPolicies_lookback_lte_6mo']/final_df['TotalPolicies_lookback']
final_df['LapsedPolicies_lookback_gt_6mo_lte_1ye'] = final_df['LapsedPolicies_lookback_gt_6mo_lte_1ye']/final_df['TotalPolicies_lookback']
final_df['LapsedPolicies_lookback_gt_1ye_lte_2ye'] = final_df['LapsedPolicies_lookback_gt_1ye_lte_2ye']/final_df['TotalPolicies_lookback']
final_df['TotalPolicies_current_Q1'] = final_df['TotalPolicies_current_Q1']/final_df['TotalPolicies_current']
final_df['TotalPolicies_current_Q2'] = final_df['TotalPolicies_current_Q2']/final_df['TotalPolicies_current']
final_df['TotalPolicies_current_Q3'] = final_df['TotalPolicies_current_Q3']/final_df['TotalPolicies_current']
final_df['TotalPolicies_current_Q4'] = final_df['TotalPolicies_current_Q4']/final_df['TotalPolicies_current']
final_df.fillna(0,inplace = True)
final_df_1.replace([np.inf, -np.inf], 0,inplace=True)

In [27]:
final_df.head()

AGENT1  YEAR                LOB  TotalPolicies_current  \
0  011199888  2010               Life                     36   
1  011199888  2011               Life                      3   
2  011199888  2011        Unit Linked                      1   
3  011199888  2014  Accident & Health                      1   
4  011199888  2014               Life                     12   

   TotalPolicies_current_Q1  TotalPolicies_current_Q2  \
0                  0.027778                       0.0   
1                  0.000000                       0.0   
2                  0.000000                       0.0   
3                  0.000000                       0.0   
4                  0.083333                       0.0   

   TotalPolicies_current_Q3  TotalPolicies_current_Q4  LapsedPolicies  \
0                  0.000000                  0.972222            35.0   
1                  0.000000                  1.000000             3.0   
2                  1.000000                  0.000000             0.0   
3                  0.000000                  1.000000             1.0   
4                  0.833333                  0.083333            10.0   

   ActiveDays  NewCustomerPolicies  #max_policies_in_a_week  NotTakenPolicies  \
0           4                 36.0                       28               0.0   
1           1                  3.0                        3               0.0   
2           1                  1.0                        1               0.0   
3           1                  1.0                        1               0.0   
4           4                 12.0                        9               0.0   

   TotalPolicies_lookback  LapsedPolicies_lookback_lte_6mo  \
0                   180.0                              0.0   
1                   196.0                              0.0   
2                    18.0                              0.0   
3                     2.0                              0.0   
4                   111.0                              0.0   

   LapsedPolicies_lookback_gt_6mo_lte_1ye  \
0                                0.033333   
1                                0.030612   
2                                0.000000   
3                                0.000000   
4                                0.081081   

   LapsedPolicies_lookback_gt_1ye_lte_2ye  LapsedPolicies_lookback_gt_2ye  \
0                                0.072222                              53   
1                                0.244898                              46   
2                                0.111111                              11   
3                                1.000000                               0   
4                                0.423423                              18   

   pol_per_day  pct_new_insured  
0          9.0              1.0  
1          3.0              1.0  
2          1.0              1.0  
3          1.0              1.0  
4          3.0              1.0

In [28]:
final_df.to_excel('AnomProfiling.xlsx',index=False)

In [87]:
df1 = df[(df['ISSUE_DATE'].dt.year>2009) & 
         (df['ISSUE_DATE'].dt.year<2017)].merge(pro[['AGENT1']].drop_duplicates(),how='inner',on='AGENT1')

In [88]:
df2 = df1.groupby(['AGENT1'])['KEY'].nunique().reset_index().rename(columns={'KEY':'TotalPols'})

In [89]:
df3 = pd.read_excel('AnomPolicies.xlsx')
df3.head()

KEY_life  Anomalous_life          KEY_AH  Anomalous_AH
0  2201_20000323             0.0  2200_PADSR8116             0
1  2201_20000494             0.0  2200_PADSR7742             0
2  2201_20003789             0.0  2200_PADSR8041             0
3  2201_20004865             0.0  2200_PADSR8047             0
4  2201_20008833             1.0  2200_PADSR7738             0

In [90]:
df4 = df1.merge(df3[df3['Anomalous_life']>0][['KEY_life','Anomalous_life']],how='inner',left_on='KEY',right_on='KEY_life')
df5 = df4.groupby(['AGENT1'])['KEY'].nunique().reset_index().rename(columns={'KEY':'AnomPols_Life'})
df6 = df2.merge(df5,how='left',on=['AGENT1']).fillna(0)

In [91]:
df6.head()

AGENT1  TotalPols  AnomPols_Life
0  011199888        105            0.0
1  022202338         74            1.0
2  032399135        116            3.0
3  032399351        128            1.0
4  055502343        169            1.0

In [92]:
df4 = df1.merge(df3[df3['Anomalous_AH']>0][['KEY_AH','Anomalous_AH']],how='inner',left_on='KEY',right_on='KEY_AH')
df5 = df4.groupby(['AGENT1'])['KEY'].nunique().reset_index().rename(columns={'KEY':'AnomPols_AH'})
df7 = df2.merge(df5,how='left',on=['AGENT1']).fillna(0)

In [93]:
df7.head()

AGENT1  TotalPols  AnomPols_AH
0  011199888        105          0.0
1  022202338         74          0.0
2  032399135        116         22.0
3  032399351        128          0.0
4  055502343        169          2.0

In [94]:
df8 = df6.merge(df7,how='inner',on=['AGENT1','TotalPols'])

In [95]:
df8.to_excel('PolicyOverlap.xlsx',index=False)

In [98]:
profile.head()

AGENT1  YEAR  CurrentYear_Accident & Health_#TotalPolicies  \
0  011199002  2010                                            23   
1  011199002  2011                                            12   
2  011199002  2012                                            34   
3  011199002  2013                                            32   
4  011199002  2014                                             7   

   CurrentYear_Life_#TotalPolicies  CurrentYear_Unit Linked_#TotalPolicies  \
0                               55                                      29   
1                               45                                      28   
2                               41                                      25   
3                               96                                      21   
4                               44                                      26   

   Lookback_Accident & Health_#TotalPolicies  Lookback_Life_#TotalPolicies  \
0                                        436                           540   
1                                        347                           522   
2                                        267                           462   
3                                        266                           478   
4                                        264                           526   

   Lookback_Unit Linked_#TotalPolicies  \
0                                  185   
1                                  207   
2                                  224   
3                                  244   
4                                  261   

   CurrentYear_Accident & Health_NewCustomer_#TotalPolicies  \
0                                           0.521739          
1                                           0.833333          
2                                           0.647059          
3                                           0.562500          
4                                           0.571429          

   CurrentYear_Life_NewCustomer_#TotalPolicies  ...    \
0                                     0.890909  ...     
1                                     0.511111  ...     
2                                     0.804878  ...     
3                                     0.927083  ...     
4                                     0.977273  ...     

   CurrentYear_Unit Linked_#avg_policy_per_active_day  \
0                                           1.450000    
1                                           1.473684    
2                                           1.190476    
3                                           1.105263    
4                                           1.300000    

   CurrentYear_by_Lkbk_Accident & Health_#avg_policy_per_active_day  \
0                                           0.556537                  
1                                           0.702017                  
2                                           1.024263                  
3                                           1.229741                  
4                                           0.777778                  

   CurrentYear_by_Lkbk_Life_#avg_policy_per_active_day  \
0                                           2.232253     
1                                           1.362069     
2                                           1.649170     
3                                           2.272627     
4                                           1.589354     

   CurrentYear_by_Lkbk_Unit Linked_#avg_policy_per_active_day  aklpe_scores  \
0                                           1.160000              -1.415124   
1                                           1.160437              -1.014774   
2                                           0.924745              -1.894692   
3                                           0.860656              -2.230084   
4                                           1.021073              -2.049830   

   aklpe  ocsvm  LOF  iforest  check  
0      0      0    0        0      0  
1      0    

In [100]:
profile['Anom'] = profile['check'].apply(lambda x: 1 if x>2 else 0)

In [103]:
profile.groupby('Anom').mean().reset_index().to_excel('AvsNACols.xlsx',index=False)